## EJERCICIO 8 - PRACTICA 3

In [197]:

 # configuración para notebook con instalación LOCAL
FUENTES_DIR  = '../Fuentes'     # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR    = '../Datos/'      # carpeta donde se encuentran los datasets
MODELOS_DIR  = '../Modelos/'    # carpeta donde se encuentran los modelos
ARCHIVOs_DIR = '../Archivos/'   # carpeta recuperar o copiar archivos

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)


In [198]:
import pandas as pd
import numpy as np
from matplotlib import pylab as plt

from sklearn import model_selection,preprocessing

from ClassPerceptron import Perceptron

In [199]:

import chardet           # para detectar la codificación de texto en archivos


nombre_archivo = DATOS_DIR + 'automobile-simple.csv' # archivo de semillas

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, encoding=result['encoding'])

# visualiza el dataframe
# print(df)
display(df)

,make,fuel-type,num-of-doors,body-style,curb-weight,engine-size,horsepower,city-mpg,highway-mpg,price,volume,eco-rating
0,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,13495.0,528019.904,33.297462
1,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,16500.0,528019.904,33.297462
2,alfa-romero,gas,two,hatchback,2823,152,154.0,19,26,16500.0,587592.640,30.898272
3,audi,gas,four,sedan,2337,109,102.0,24,30,13950.0,634816.956,42.697819
4,audi,gas,four,sedan,2824,136,115.0,18,22,17450.0,636734.832,27.997459
...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,four,sedan,2952,141,114.0,23,28,16845.0,721961.760,38.354298
201,volvo,gas,four,sedan,3049,141,160.0,19,25,19045.0,720913.920,32.043335
202,volvo,gas,four,sedan,3012,173,134.0,18,23,21485.0,721961.760,29.806494
203,volvo,diesel,four,sedan,3217,145,106.0,26,27,22470.0,721961.760,35.073342


In [200]:
# elimino las filas con datos faltantes
df = df.dropna()

# verifico que no hayan columnas con datos faltantes (porque elimine las filas que los tenian)
df.isnull().sum()


make            0
fuel-type       0
num-of-doors    0
body-style      0
curb-weight     0
engine-size     0
horsepower      0
city-mpg        0
highway-mpg     0
price           0
volume          0
eco-rating      0
dtype: int64

In [201]:
# calculo la media de columna eco-rating:
media = df['eco-rating'].mean()
print(media)


# Agregar la nueva columna binaria
df['auto-ECOLOGICO'] = (df['eco-rating'] > media).astype(int)

# Mostrar el DataFrame resultante
print(df)

42.09494272062433
            make fuel-type num-of-doors   body-style  curb-weight  \
0    alfa-romero       gas          two  convertible         2548   
1    alfa-romero       gas          two  convertible         2548   
2    alfa-romero       gas          two    hatchback         2823   
3           audi       gas         four        sedan         2337   
4           audi       gas         four        sedan         2824   
..           ...       ...          ...          ...          ...   
200        volvo       gas         four        sedan         2952   
201        volvo       gas         four        sedan         3049   
202        volvo       gas         four        sedan         3012   
203        volvo    diesel         four        sedan         3217   
204        volvo       gas         four        sedan         3062   

     engine-size  horsepower  city-mpg  highway-mpg    price      volume  \
0            130       111.0        21           27  13495.0  528019.904   
1

### numerizo atributos no numericos:

In [203]:
# Mapear los valores categóricos a números
df['fuel-type'] = df['fuel-type'].map({'diesel': 0, 'gas': 1})

# num-of-doors:
df['num-of-doors'] = df['num-of-doors'].map({'two': 2, 'three': 3, 'four': 4, 'five': 5})

# body-style:
df['body-style'] = df['body-style'].map({'convertible': 1, 'hatchback': 2, 'sedan': 2})

print(df)

            make  fuel-type  num-of-doors  body-style  curb-weight  \
0    alfa-romero          1             2         1.0         2548   
1    alfa-romero          1             2         1.0         2548   
2    alfa-romero          1             2         2.0         2823   
3           audi          1             4         2.0         2337   
4           audi          1             4         2.0         2824   
..           ...        ...           ...         ...          ...   
200        volvo          1             4         2.0         2952   
201        volvo          1             4         2.0         3049   
202        volvo          1             4         2.0         3012   
203        volvo          0             4         2.0         3217   
204        volvo          1             4         2.0         3062   

     engine-size  horsepower  city-mpg  highway-mpg    price      volume  \
0            130       111.0        21           27  13495.0  528019.904   
1      

In [262]:
# -- Columna fuel-type --
media = df['fuel-type'].mean()
desvio = df['fuel-type'].std()
df['fuel-type']= (df['fuel-type']-media)/desvio

# -- Columna num-of-doors --
media = df['num-of-doors'].mean()
desvio = df['num-of-doors'].std()
df['num-of-doors']= (df['num-of-doors']-media)/desvio

# -- Columna body-style --
media = df['body-style'].mean()
desvio = df['body-style'].std()
df['body-style']= (df['body-style']-media)/desvio

# -- Columna curb-weight --
media = df['curb-weight'].mean()
desvio = df['curb-weight'].std()
df['curb-weight']= (df['curb-weight']-media)/desvio

# -- Columna engine-size --
media = df['engine-size'].mean()
desvio = df['engine-size'].std()
df['engine-size']= (df['engine-size']-media)/desvio

# -- Columna horsepower --
media = df['horsepower'].mean()
desvio = df['horsepower'].std()
df['horsepower']= (df['horsepower']-media)/desvio

# -- Columna city-mpg --
media = df['city-mpg'].mean()
desvio = df['city-mpg'].std()
df['city-mpg']= (df['city-mpg']-media)/desvio

# -- Columna highway-mpg --
media = df['highway-mpg'].mean()
desvio = df['highway-mpg'].std()
df['highway-mpg']= (df['highway-mpg']-media)/desvio

# -- Columna price --
media = df['price'].mean()
desvio = df['price'].std()
df['price']= (df['price']-media)/desvio

# -- Columna volume --
media = df['volume'].mean()
desvio = df['volume'].std()
df['volume']= (df['volume']-media)/desvio

# -- Columna eco-rating --
media = df['eco-rating'].mean()
desvio = df['eco-rating'].std()
df['eco-rating']= (df['eco-rating']-media)/desvio


print(df)
df.isnull().sum()
df = df.dropna()
df.isnull().sum()
#round(df[['Age', 'AgeLineal', 'AgeNorm']].describe(),4)

            make  fuel-type  num-of-doors  body-style  curb-weight  \
0    alfa-romero   0.325883     -1.144971   -5.132192    -0.020041   
1    alfa-romero   0.325883     -1.144971   -5.132192    -0.020041   
2    alfa-romero   0.325883     -1.144971    0.193668     0.506999   
3           audi   0.325883      0.868951    0.193668    -0.424424   
4           audi   0.325883      0.868951    0.193668     0.508916   
..           ...        ...           ...         ...          ...   
200        volvo   0.325883      0.868951    0.193668     0.754229   
201        volvo   0.325883      0.868951    0.193668     0.940130   
202        volvo   0.325883      0.868951    0.193668     0.869220   
203        volvo  -3.053009      0.868951    0.193668     1.262104   
204        volvo   0.325883      0.868951    0.193668     0.965045   

     engine-size  horsepower  city-mpg  highway-mpg     price    volume  \
0       0.071698    0.196496 -0.644979    -0.530911  0.026885 -1.150167   
1       0

make              0
fuel-type         0
num-of-doors      0
body-style        0
curb-weight       0
engine-size       0
horsepower        0
city-mpg          0
highway-mpg       0
price             0
volume            0
eco-rating        0
auto-ECOLOGICO    0
dtype: int64

### Division en X e Y

In [265]:
datos  = np.array(df)
Y = datos[:,12].reshape(-1,1) #variable que quiero predecir
X = np.delete(datos, [12], 1) #elimino la variable target

# elimino columna "make" (no da datos al modelo)
X = np.delete(X, [0], 1)  # Elimina la primera columna (índice 0)

#convierto los datos de los arreglos en flotante (sino me da error)
X = X.astype(float)
Y = Y.astype(float)


### Normalizacion

In [268]:
normalizador = preprocessing.StandardScaler()
# X = normalizador.fit_transform(X)

## Entrenamiento utilizando salida sigmoide (sigmoid) para la neurona

In [271]:
from ClassNeuronaGral import NeuronaGradiente
modelo_sigmoid = NeuronaGradiente(alpha=0.01, n_iter=100, FUN='sigmoid', COSTO='EC_binaria')

In [273]:
modelo_sigmoid.fit(X,Y)

In [275]:
pred  = modelo_sigmoid.predict(X)
modelo_sigmoid.errors_

[array([93.28411004]),
 array([71.61208219]),
 array([61.30984956]),
 array([55.09114806]),
 array([50.82017868]),
 array([47.64641788]),
 array([45.15977676]),
 array([43.1365772]),
 array([41.44356096]),
 array([39.99586711]),
 array([38.73656741]),
 array([37.62584668]),
 array([36.63489425]),
 array([35.74226465]),
 array([34.93161007]),
 array([34.19021297]),
 array([33.50800522]),
 array([32.87689376]),
 array([32.29028549]),
 array([31.74274527]),
 array([31.22974504]),
 array([30.74747651]),
 array([30.29270926]),
 array([29.86268178]),
 array([29.45501662]),
 array([29.06765369]),
 array([28.69879731]),
 array([28.34687358]),
 array([28.01049604]),
 array([27.68843754]),
 array([27.37960713]),
 array([27.08303105]),
 array([26.79783671]),
 array([26.52323944]),
 array([26.25853121]),
 array([26.00307112]),
 array([25.75627733]),
 array([25.51762011]),
 array([25.28661588]),
 array([25.06282209]),
 array([24.84583278]),
 array([24.63527468]),
 array([24.4308039]),
 array([24.23

## Entrenamiento utilizando salida tangente hiperbólico (tanh) para la neurona

In [214]:
modelo_tanh = NeuronaGradiente(alpha=0.01, n_iter=100, FUN='tanh', COSTO='EC_binaria')

In [215]:
#modelo_tanh.fit(X,Y)
#modelo_tanh._w=


In [216]:
#pred = modelo_tanh.predict(X)
#pred

In [217]:
#PREGUNTAR: POR QUE ME DA TODO 0 Y TODO -1 !!!!!!!!!!!!!!!!!!!!1